god damn model is start


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import model_selection

In [2]:
df = pd.read_csv('cleanedManuel_data.csv')

In [3]:
print(df.columns)

Index(['location', 'districk', 'neighborhood', 'room', 'living_room', 'area',
       'age', 'floor', 'price'],
      dtype='object')


In [4]:
X = df.drop(columns=['price'])
y = df['price']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#grid search
params = {"colsample_bytree":[0.4,0.5,0.6],
         "learning_rate":[0.01,0.02,0.09],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[100,200,500,2000]}

In [7]:

xgb = XGBRegressor()


In [8]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)


In [10]:
df['location'] = df['location'].astype('category')
df['districk'] = df['districk'].astype('category')
df['neighborhood'] = df['neighborhood'].astype('category')
df['room'] = df['room'].astype('int')
df['living_room'] = df['living_room'].astype('int')
df['area'] = df['area'].astype('int')
df['age'] = df['age'].astype('int')
df['floor'] = df['floor'].astype('int')
df['price'] = df['price'].astype('int')



In [11]:
print(X_train.dtypes)
print(y_train.dtypes)
print(X_train.isnull().sum())
print(y_train.isnull().sum())

location        object
districk        object
neighborhood    object
room             int64
living_room      int64
area             int64
age              int64
floor            int64
dtype: object
int64
location        0
districk        0
neighborhood    0
room            0
living_room     0
area            0
age             0
floor           0
dtype: int64
0


In [12]:
X_train = X_train.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)


In [13]:
# objeler
object_cols = X_train.select_dtypes(include=['object']).columns
print("Object sütunlar:", object_cols)

# Bu sütunları one-hot encoding ile sayısal hale getir
X_train_encoded = pd.get_dummies(X_train, drop_first=True)


Object sütunlar: Index(['location', 'districk', 'neighborhood'], dtype='object')


In [14]:
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Train ve test verilerini aynı sütun yapısına hizala
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)


In [15]:
grid.fit(X_train_encoded, y_train)


Fitting 10 folds for each of 180 candidates, totalling 1800 fits


KeyboardInterrupt: 

In [95]:
grid.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [96]:
#grid search
params = {"colsample_bytree":[0.1,0.2,0.4,0.5],
         "learning_rate":[0.1,1,5,10],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[500,2000,5000,10000]}

In [16]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)


In [17]:
grid.fit(X_train_encoded, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


KeyboardInterrupt: 

In [34]:
grid.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 2000}

In [18]:
##en uygun parametreler
xgb1 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.09, max_depth=4, n_estimators=2000)

In [19]:
#model eğitimi
model_xgb = xgb1.fit(X_train_encoded, y_train)

In [20]:
model_xgb.predict(X_test_encoded[15:21])

array([45610.38 , 29191.479, 18717.32 , 35062.594, 15774.019, 43186.832],
      dtype=float32)

In [21]:
y_test[15:21]

6653    35000
2465    23000
4615    16000
3151    42000
132     16750
2206    20000
Name: price, dtype: int64

In [22]:
model_xgb.score(X_test_encoded, y_test)

0.7117846608161926

In [23]:
model_xgb.score(X_train_encoded, y_train)

0.8769660592079163

In [24]:
print(model_xgb.predict(X_test_encoded))

[28538.701 20926.611 12081.798 ... 86561.22  42976.582 23319.35 ]


In [38]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

# Veriyi oku
df = pd.read_csv('cleanedManuel_data.csv')

# Kategorik ve sayısal sütunları ayarla
df['location'] = df['location'].astype('category')
df['districk'] = df['districk'].astype('category')
df['neighborhood'] = df['neighborhood'].astype('category')
df['room'] = df['room'].astype('int')
df['living_room'] = df['living_room'].astype('int')
df['area'] = df['area'].astype('int')
df['age'] = df['age'].astype('int')
df['floor'] = df['floor'].astype('int')
df['price'] = df['price'].astype('int')

# X ve y'yi ayır
X = df.drop(columns=['price'])
y = df['price']

# Eğitim ve test verisi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kategorik değişkenleri kodla
X_train = X_train.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)
X_test = X_test.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)

# One-hot encoding
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Modeli oluştur ve eğit
xgb1 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.09, max_depth=4, n_estimators=2000)
model_xgb = xgb1.fit(X_train_encoded, y_train)

# Eğitimdeki kategorik değerlerden örnek al
ornek_veri = {
    'location': df['location'].cat.categories[0],  # Eğitimdeki ilk kategori
    'districk': df['districk'].cat.categories[0],
    'neighborhood': df['neighborhood'].cat.categories[0],
    'room': 3,
    'living_room': 1,
    'area': 120,
    'age': 15,
    'floor': 1
}
ornek_df = pd.DataFrame([ornek_veri])

# Kategorik değişkenleri kodla
for col in ['location', 'districk', 'neighborhood']:
    ornek_df[col] = ornek_df[col].astype('category')
    ornek_df[col] = ornek_df[col].cat.set_categories(df[col].cat.categories)

# Diğer sütunları int'e çevir
for col in ['room', 'living_room', 'area', 'age', 'floor']:
    ornek_df[col] = ornek_df[col].astype(int)

# Kodlama ve one-hot işlemleri
ornek_df = ornek_df.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)
ornek_encoded = pd.get_dummies(ornek_df, drop_first=True)
ornek_encoded = ornek_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Tahmin al
tahmin = model_xgb.predict(ornek_encoded)
print("Tahmin edilen fiyat:", tahmin[0])

Tahmin edilen fiyat: 41699.016
